## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Cloudiness,Wind Speed,Description
0,0,Cape Town,ZA,-33.9258,18.4232,61.11,75,4.61,fog
1,1,Ushuaia,AR,-54.8000,-68.3000,36.12,83,6.80,broken clouds
2,2,Kieta,PG,-6.2167,155.6333,85.05,96,4.36,overcast clouds
3,3,Constitucion,CL,-35.3333,-72.4167,54.64,25,12.62,scattered clouds
4,4,Mataura,NZ,-46.1927,168.8643,56.26,75,3.49,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Cloudiness,Wind Speed,Description
0,0,Cape Town,ZA,-33.9258,18.4232,61.11,75,4.61,fog
3,3,Constitucion,CL,-35.3333,-72.4167,54.64,25,12.62,scattered clouds
4,4,Mataura,NZ,-46.1927,168.8643,56.26,75,3.49,broken clouds
5,5,Busselton,AU,-33.6500,115.3333,58.41,0,9.33,clear sky
10,10,Chinsali,ZM,-10.5414,32.0816,61.27,78,4.27,broken clouds
11,11,Saint George,US,37.1041,-113.5841,58.62,0,12.66,clear sky
12,12,Albany,US,42.6001,-73.9662,59.41,23,6.17,few clouds
14,14,Mar Del Plata,AR,-38.0023,-57.5575,57.09,0,8.05,clear sky
18,18,East London,ZA,-33.0153,27.9116,67.98,100,2.21,light rain
20,20,New Norfolk,AU,-42.7826,147.0587,65.71,23,5.01,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        264
City           264
Country        261
Lat            264
Lng            264
Max Temp       264
Cloudiness     264
Wind Speed     264
Description    264
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna(how='any')
clean_preferred_cities_df.count()

City_ID        261
City           261
Country        261
Lat            261
Lng            261
Max Temp       261
Cloudiness     261
Wind Speed     261
Description    261
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()
                                     
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,61.11,fog,-33.9258,18.4232,
3,Constitucion,CL,54.64,scattered clouds,-35.3333,-72.4167,
4,Mataura,NZ,56.26,broken clouds,-46.1927,168.8643,
5,Busselton,AU,58.41,clear sky,-33.6500,115.3333,
10,Chinsali,ZM,61.27,broken clouds,-10.5414,32.0816,
11,Saint George,US,58.62,clear sky,37.1041,-113.5841,
12,Albany,US,59.41,few clouds,42.6001,-73.9662,
14,Mar Del Plata,AR,57.09,clear sky,-38.0023,-57.5575,
18,East London,ZA,67.98,light rain,-33.0153,27.9116,
20,New Norfolk,AU,65.71,few clouds,-42.7826,147.0587,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
       print("Hotel not found... skipping.")    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,61.11,fog,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
3,Constitucion,CL,54.64,scattered clouds,-35.3333,-72.4167,Hotel Boutique Las Azucenas
4,Mataura,NZ,56.26,broken clouds,-46.1927,168.8643,Ellie's Villa
5,Busselton,AU,58.41,clear sky,-33.6500,115.3333,Observatory Guest House
10,Chinsali,ZM,61.27,broken clouds,-10.5414,32.0816,Chisakatu Guest House
...,...,...,...,...,...,...,...
739,Port Shepstone,ZA,69.82,clear sky,-30.7414,30.4550,My Den Beachfront Bed and Breakfast | Self-Cat...
740,Manjo,CM,71.83,heavy intensity rain,4.8428,9.8217,Hôtel la Grace
741,Kununurra,AU,73.38,light rain,-15.7667,128.7333,Hotel Kununurra
750,Copiapo,CL,58.69,clear sky,-27.3667,-70.3333,Antay Casino Hotel


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))